In [69]:
import re
from tabulate import tabulate
from collections import deque

class CacheCoherenceVisualizer:
    def __init__(self):
        self.nodes = {
            1: {"state": "Proc_I", "request": {"incoming": [], "outgoing": []}, 
                "response": {"incoming": [], "outgoing": []}, 
                "forward": {"incoming": [], "outgoing": []}},
            2: {"state": "Proc_I", "request": {"incoming": [], "outgoing": []}, 
                "response": {"incoming": [], "outgoing": []}, 
                "forward": {"incoming": [], "outgoing": []}},
            3: {"state": "Proc_I", "request": {"incoming": [], "outgoing": []}, 
                "response": {"incoming": [], "outgoing": []}, 
                "forward": {"incoming": [], "outgoing": []}},
            7: {"state": "Dir_I", "request": {"incoming": [], "outgoing": []}, 
                "response": {"incoming": [], "outgoing": []}, 
                "forward": {"incoming": [], "outgoing": []}},
        }
        self.snapshots = deque()  # Store snapshots for navigation
        self.current_index = -1  # Track current snapshot

    def parse_log_file(self, file_path):
        """Read and parse the log from a file."""
        try:
            with open(file_path, "r") as file:
                log = file.read()
            self.parse_log(log)
        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
        except Exception as e:
            print(f"Error: {e}")

    def parse_log(self, log):
        pattern = r"(Create|Receive) Msg (\d+):: type: (\w+), src: (\d+), dst: (\d+), .*?src_state: (\w+), dst_state: (\w+)"
        matches = re.finditer(pattern, log)

        for match in matches:
            action, msg_id, msg_type, src, dst, src_state, dst_state = match.groups()
            src, dst = int(src), int(dst)
            msg_id = int(msg_id)

            # Print the log entry
            print(f"Log Entry: {match.group(0)}")

            if action == "Create":
                self.create_message(msg_id, msg_type, src, dst, src_state, dst_state)
            elif action == "Receive":
                self.receive_message(msg_id, msg_type, src, dst, src_state, dst_state)
        self.current_index = 0  # Track current snapshot

    def create_message(self, msg_id, msg_type, src, dst, src_state, dst_state):
        channel = self.get_channel(msg_type)
        # Add message to the outgoing list of the source node
        self.nodes[src][channel]["outgoing"].append({"id": msg_id, "type": msg_type, "dst": dst})
        # Update source and destination states
        self.nodes[src]["state"] = src_state
        self.nodes[dst]["state"] = dst_state
        # Take a snapshot
        self.take_snapshot(f"Create Msg {msg_id}: {msg_type} from {src} to {dst}")

    def receive_message(self, msg_id, msg_type, src, dst, src_state, dst_state):
        channel = self.get_channel(msg_type)
        # Remove the corresponding message from the source's outgoing list
        outgoing = self.nodes[src][channel]["outgoing"]
        message = next((m for m in outgoing if m["id"] == msg_id), None)
        if message:
            outgoing.remove(message)
        # Add message to the destination's incoming list
        self.nodes[dst][channel]["incoming"] = [{"id": msg_id, "type": msg_type, "src": src}]
        # Update source and destination states
        self.nodes[src]["state"] = src_state
        self.nodes[dst]["state"] = dst_state
        # Take a snapshot
        self.take_snapshot(f"Receive Msg {msg_id}: {msg_type} from {src} to {dst}")

    def get_channel(self, msg_type):
        if msg_type in {"GetS", "GetM", "PutS", "PutM"}:
            return "request"
        elif msg_type in {"Data", "NACK", "PutAck", "InvAck", "FwdAck"}:
            return "response"
        elif msg_type in {"Inv", "FwdGetS", "FwdGetM"}:
            return "forward"
        else:
            raise ValueError("Invalid message type")

    def take_snapshot(self, header):
        snapshot = {
            "header": header,
            "nodes": {k: v.copy() for k, v in self.nodes.items()},
        }
        self.snapshots.append(snapshot)
        self.current_index = len(self.snapshots) - 1

    def previous(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.display_snapshot()

    def next(self):
        if self.current_index < len(self.snapshots) - 1:
            self.current_index += 1
            self.display_snapshot()

    def display_snapshot(self):
        snapshot = self.snapshots[self.current_index]
        print(f"\n=== {snapshot['header']} ===\n")

        # Node titles for side-by-side display
        node_titles = ["Proc_1", "Proc_2", "Proc_3", "Dir"]
        node_ids = [1, 2, 3, 7]

        # Print the titles for nodes
        print("   ".join(f"{title:<30}" for title in node_titles))
        print("=" * 80)

        # Loop through channels (request, response, forward)
        for channel_name in ["request", "response", "forward"]:
            print(f"{channel_name.capitalize()}" + " Outgoing")

            # Prepare outgoing rows for each node
            outgoing_rows = [
                tabulate(
                    snapshot["nodes"][node_id][channel_name]["outgoing"],
                    headers="keys",
                    tablefmt="plain",
                    showindex=False
                ) or "[Empty]"
                for node_id in node_ids
            ]

            # Print outgoing lists side-by-side
            self.print_side_by_side(outgoing_rows)

            # Prepare incoming rows for each node
            print(f"{channel_name.capitalize()}" + " Incoming")
            incoming_rows = [
                tabulate(
                    snapshot["nodes"][node_id][channel_name]["incoming"],
                    headers="keys",
                    tablefmt="plain",
                    showindex=False
                ) or "[Empty]"
                for node_id in node_ids
            ]

            # Print incoming lists side-by-side
            self.print_side_by_side(incoming_rows)
            print("-" * 80)

    def print_side_by_side(self, tables):
        """Helper function to print multiple tables side by side."""
        table_lines = [table.splitlines() for table in tables]
        max_lines = max(len(lines) for lines in table_lines)
        padded_tables = [
            lines + [""] * (max_lines - len(lines)) for lines in table_lines
        ]
        for row in zip(*padded_tables):
            print("   ".join(f"{col:<30}" for col in row))


    def format_list(self, lst):
        """Format the list to make it compact and readable."""
        if not lst:
            return "[]"
        return "\n".join(f"{item['id']}:{item['type']}" for item in lst)


visualizer = CacheCoherenceVisualizer()
visualizer.parse_log_file('msi_opt_sim_preserve.log')
# visualizer.previous()  # Navigate to previous snapshot
# visualizer.next()      # Navigate to next snapshot


Log Entry: Create Msg 0:: type: GetM, src: 3, dst: 7, ack_cnt: 0, src_state: Proc_IM_D, dst_state: Dir_I
Log Entry: Create Msg 1:: type: GetS, src: 1, dst: 7, ack_cnt: 0, src_state: Proc_IS_D, dst_state: Dir_I
Log Entry: Create Msg 2:: type: GetS, src: 2, dst: 7, ack_cnt: 0, src_state: Proc_IS_D, dst_state: Dir_I
Log Entry: Receive Msg 2:: type: GetS, src: 2, dst: 7, ack_cnt: 0, src_state: Proc_IS_D, dst_state: Dir_I
Log Entry: Create Msg 3:: type: Data, src: 7, dst: 2, ack_cnt: 0, src_state: Dir_S, dst_state: Proc_IS_D
Log Entry: Receive Msg 0:: type: GetM, src: 3, dst: 7, ack_cnt: 0, src_state: Proc_IM_D, dst_state: Dir_S
Log Entry: Create Msg 4:: type: Data, src: 7, dst: 3, ack_cnt: 1, src_state: Dir_SM_A, dst_state: Proc_IM_D
Log Entry: Receive Msg 4:: type: Data, src: 7, dst: 3, ack_cnt: 1, src_state: Dir_SM_A, dst_state: Proc_IM_D
Log Entry: Create Msg 5:: type: PutM, src: 3, dst: 7, ack_cnt: 0, src_state: Proc_MI_A, dst_state: Dir_SM_A
Log Entry: Receive Msg 1:: type: GetS, src:

In [70]:
visualizer.next()


=== Create Msg 1: GetS from 1 to 7 ===

Proc_1                           Proc_2                           Proc_3                           Dir                           
Request Outgoing
  id  type      dst              [Empty]                          [Empty]                          [Empty]                       
   7  GetS        7                                                                                                              
Request Incoming
[Empty]                          [Empty]                          [Empty]                            id  type      src           
                                                                                                      5  PutM        3           
--------------------------------------------------------------------------------
Response Outgoing
[Empty]                          [Empty]                          [Empty]                          [Empty]                       
Response Incoming
  id  type      src         